In [1]:
import sys
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install plotly
!{sys.executable} -m pip install kaleido
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install pyopencl
!{sys.executable} -m pip install nbformat


Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip3 install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip3 install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip3 install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip3 install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip3 install --upgrade pip
Defaulting to user installation because normal sit

In [6]:

# Second cell - fix imports
import numpy as np
import pandas as pd  # Add pandas import
import plotly.express as px
import os
import plotly.graph_objects as g

In [10]:
def visualize(csv_path: str):
    '''visualizes cub, futhark and our impl of radix-sort'''
    df = pd.read_csv(csv_path)
    
    # Map data types to their sizes in bytes
    dtype_sizes = {
        'u8': 1,
        'u16': 2,
        'u32': 4,
        'u64': 8
    }
    
    # futhark outputs milliseconds not microseconds so we need to adjust
    print("futh before", df[df['impl'] == 'futhark']['microseconds'])
    df.where(df['impl']=='futhark')['microseconds'].apply(lambda elm: elm*1000)
    print("fut after", df[df['impl'] == 'futhark']['microseconds'])
    
    # Calculate GB/s based on data type
    df['GB/s'] = df.apply(
        lambda row: (row['dataset_size'] * dtype_sizes[row['data_type']]) / 
                   (row['microseconds'] * 1e6 * 1e9),
        axis=1
    )
    
    # Create figure with dual y-axes
    fig = g.Figure()
    
    # Create separate traces for each implementation and data type
    for impl in df['impl'].unique():
        for dtype in df['data_type'].unique():
            mask = (df['impl'] == impl) & (df['data_type'] == dtype)
            name = f"{impl} ({dtype})"
            
            fig.add_trace(g.Scatter(
                x=df[mask]['dataset_size'],
                y=df[mask]['GB/s'],
                mode='lines+markers',
                name=name
            ))
    
    fig.update_layout(
        title='Radix Sort Performance',
        xaxis_title='Dataset Size',
        yaxis_title='Throughput (GB/s)',
        hovermode='x unified',
        xaxis=dict(type='log'),  # Use log scale for dataset size
    )
    
    return fig


def save_fig(fig : g.Figure, name: str):
    current_dir = os.getcwd()
    os.chdir('..') # move from src to root
    fig.write_png(f'images/{name}.png')

fig = visualize('benchmarks.csv')

save_fig(fig, 'bench')


FileNotFoundError: [Errno 2] No such file or directory: 'benchmarks.csv'